In [1]:
%%bash
git clone https://github.com/ZhuoqunWang0120/cs671kaggle.git

Cloning into 'cs671kaggle'...


In [5]:
!pip install pycaret

     |████████████████████████████████| 288 kB 5.0 MB/s 
     |████████████████████████████████| 1.7 MB 33.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 58.8 MB/s 
     |████████████████████████████████| 6.8 MB 34.3 MB/s 
     |████████████████████████████████| 15.5 MB 48.5 MB/s 
     |████████████████████████████████| 2.0 MB 18.9 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 113 kB 51.9 MB/s 
     |████████████████████████████████| 1.3 MB 44.1 MB/s 
     |████████████████████████████████| 261 kB 59.2 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
     |████████████████████████████████| 303 kB 54.3 MB/s 
     |████████████████████████████████| 675 kB 61.5 MB/s 
     |█████████████████

In [60]:
import os
from sklearn import model_selection
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve
import pycaret
from pycaret.classification import *


In [3]:
os.chdir('./cs671kaggle')

In [62]:
train_data = pd.read_csv('data/train.csv')
train_data = train_data.dropna()

In [63]:
train_data.head()

,id,Decision,Driving_to,Passanger,Weather,Temperature,Time,Coupon,Coupon_validity,Gender,Age,Maritalstatus,Children,Education,Occupation,Income,Bar,Coffeehouse,Carryaway,Restaurantlessthan20,Restaurant20to50,Direction_same,Distance
0,1,1,No Urgent Place,Partner,Sunny,80,6PM,Bar,1d,Male,31,Married partner,0,Some college - no degree,Construction & Extraction,$100000 or More,2.0,0.0,3.0,2.0,2.0,0,2
1,2,0,Work,Alone,Sunny,80,7AM,Restaurant(20-50),1d,Female,26,Married partner,1,Some college - no degree,Unemployed,$62500 - $74999,0.0,1.0,4.0,3.0,0.0,0,2
2,3,1,No Urgent Place,Partner,Sunny,80,6PM,Restaurant(<20),1d,Male,50plus,Married partner,1,Associates degree,Unemployed,$37500 - $49999,1.0,1.0,1.0,3.0,1.0,0,1
3,4,1,Work,Alone,Sunny,55,7AM,Restaurant(<20),1d,Male,31,Married partner,0,Some college - no degree,Construction & Extraction,$100000 or More,2.0,0.0,3.0,2.0,2.0,0,1
4,5,1,Home,Alone,Sunny,30,6PM,Carry out & Take away,2h,Male,41,Single,0,Some college - no degree,Arts Design Entertainment Sports & Media,$12500 - $24999,0.0,0.0,2.0,2.0,0.0,1,2


In [64]:
train_data.dtypes

id                        int64
Decision                  int64
Driving_to               object
Passanger                object
Weather                  object
Temperature               int64
Time                     object
Coupon                   object
Coupon_validity          object
Gender                   object
Age                      object
Maritalstatus            object
Children                  int64
Education                object
Occupation               object
Income                   object
Bar                     float64
Coffeehouse             float64
Carryaway               float64
Restaurantlessthan20    float64
Restaurant20to50        float64
Direction_same            int64
Distance                  int64
dtype: object

In [65]:
train_data = train_data.drop(columns = 'id')
train_data.describe()

,Decision,Temperature,Children,Bar,Coffeehouse,Carryaway,Restaurantlessthan20,Restaurant20to50,Direction_same,Distance
count,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000,9689.000000
mean,0.570337,63.406440,0.408401,1.043451,1.574363,2.422644,2.297760,1.268965,0.211374,1.684281
std,0.495053,19.106122,0.491563,1.093139,1.239517,0.930395,0.918318,0.882851,0.408304,0.675697
min,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,55.000000,0.000000,0.000000,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000
50%,1.000000,80.000000,0.000000,1.000000,1.000000,2.000000,2.000000,1.000000,0.000000,2.000000
75%,1.000000,80.000000,1.000000,2.000000,2.000000,3.000000,3.000000,2.000000,0.000000,2.000000
max,1.000000,80.000000,1.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000,3.000000


In [66]:
train_data.iloc[:, [i for i in range(train_data.shape[1]) if train_data.dtypes[i] == object]].apply(lambda x: x.dropna().value_counts().count())

Driving_to          3
Passanger           4
Weather             3
Time                5
Coupon              5
Coupon_validity     2
Gender              2
Age                 8
Maritalstatus       5
Education           6
Occupation         25
Income              9
dtype: int64

In [67]:
# transform age to age groups
train_data.Age = pd.Series(list(map(lambda s: s[0], train_data.Age.astype('str'))))
# transform dtype of temperature
train_data.Temperature = train_data.Temperature.astype('float')
# one-hot encoding
train_data = pd.get_dummies(train_data,dummy_na=False)
Xtrain, ytrain = train_data.drop(columns = 'Decision'), train_data.Decision

In [68]:
models = setup(data = train_data, target = "Decision", session_id = 0)

,Description,Value
0,session_id,0
1,Target,Decision
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(9689, 84)"
5,Missing Values,False
6,Numeric Features,80
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [69]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7417,0.8014,0.8220,0.7496,0.7838,0.4647,0.4684,1.133
lightgbm,Light Gradient Boosting Machine,0.7400,0.8047,0.8238,0.7462,0.7829,0.4609,0.4647,0.189
et,Extra Trees Classifier,0.7263,0.7880,0.7945,0.7434,0.7678,0.4355,0.4375,1.219
gbc,Gradient Boosting Classifier,0.7216,0.7791,0.8207,0.7263,0.7705,0.4202,0.4255,1.097
lr,Logistic Regression,0.6825,0.7292,0.7688,0.7022,0.7339,0.3426,0.3449,0.802
ada,Ada Boost Classifier,0.6818,0.7339,0.7766,0.6984,0.7354,0.3393,0.3424,0.376
lda,Linear Discriminant Analysis,0.6808,0.7290,0.7652,0.7015,0.7318,0.3394,0.3415,0.121
ridge,Ridge Classifier,0.6803,0.0000,0.7663,0.7006,0.7318,0.3382,0.3405,0.028
knn,K Neighbors Classifier,0.6685,0.6992,0.7432,0.6956,0.7184,0.3166,0.3179,0.547
dt,Decision Tree Classifier,0.6593,0.6536,0.6952,0.7034,0.6989,0.3063,0.3069,0.071
